In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
%%time
import static_individual_estimates
static_individual_estimates.estimate_paces(ve_or_ju)

2019-11-06 20:45:06,024 None
2019-11-06 20:45:06,095 None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14740 entries, 0 to 14739
Data columns (total 21 columns):
mean_team_id           14740 non-null float64
teams                  14740 non-null object
name                   14740 non-null object
num_runs               14740 non-null int64
num_valid_times        14740 non-null int64
mean_pace              14338 non-null float64
stdev                  14338 non-null float64
log_stdev              14740 non-null float64
most_common_leg        14740 non-null int64
most_common_country    14740 non-null object
pace_1                 14338 non-null float64
pace_2                 8391 non-null float64
pace_3                 6004 non-null float64
pace_4                 4264 non-null float64
pace_5                 3060 non-null float64
pace_6                 2093 non-null float64
pace_7                 1258 non-null float64
pace_8                 493 non-null float64
leg_nro                14740 non-null int64
team_id                14740 non-nul

2019-11-06 20:45:06,406 []
2019-11-06 20:45:06,416 
(14740, 41)
2019-11-06 20:45:06,882     log_q_low  predicted  log_q_high  log_std
0       1.847      6.867       2.032    0.092
1       1.835      7.057       2.061    0.113
2       1.843      7.133       2.072    0.115
3       1.816      6.904       2.035    0.109
4       1.958      8.013       2.177    0.110
5       1.807      6.564       2.024    0.109
6       1.830      7.050       2.059    0.115
7       1.823      6.586       2.029    0.103
8       1.837      7.054       1.962    0.063
9       1.865      7.034       2.089    0.112
10      1.799      6.763       2.061    0.131
11      1.867      6.966       2.038    0.086
12      1.814      6.757       2.011    0.098
13      1.898      7.599       2.150    0.126
14      1.834      7.144       2.090    0.128
2019-11-06 20:45:06,886 0.18015259754165835
2019-11-06 20:45:06,892 prior_mean_error mean: 1.452
2019-11-06 20:45:06,894 prior_mean_error_in_sd mean: 1.195
/Users/oskari/.virtu

CPU times: user 1.01 s, sys: 75.4 ms, total: 1.08 s
Wall time: 1.12 s


In [3]:
%%time
static_individual_estimates.combine_estimates_with_running_order(ve_or_ju)

2019-11-06 20:45:07,111 
(6759, 7)
2019-11-06 20:45:07,141 
(14740, 12)


Found 1 history with switched name zhanalova barbora 
name 'sanna timonen' team_name 'MAASOTAKOULU LPR'
by_name 0 by_name_and_colon 5 runners 2
Duplicate runner       Unnamed: 0  mean_team_id  num_valid_times  mean_pace  stdev  log_stdev  \
7141        7141         501.0                1     13.403    0.0        0.0   
7144        7144         777.0                1     10.631    0.0        0.0   

      prior_mean  prior_log_std  predicted_log_pace_mean  \
7141     10.8707         0.1739                   2.5955   
7144     12.9885         0.2035                   2.3638   

      predicted_log_pace_std                                   name  \
7141                     0.0         sanna timonen:MAASOTAKOULU LPR   
7144                     0.0  sanna timonen:MAASOTAKOULU LPR VENLAT   

                        teams  num_runs  
7141         MAASOTAKOULU LPR         1  
7144  MAASOTAKOULU LPR VENLAT         1  
Found 1 history with switched name hjelseth lerke 
Found 1 history with switc

2019-11-06 20:45:57,087 None
2019-11-06 20:45:57,117 None
2019-11-06 20:45:57,257 []


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6759 entries, 0 to 6758
Data columns (total 12 columns):
team_id           6759 non-null int64
team              6759 non-null object
team_base_name    6759 non-null object
team_country      6759 non-null object
leg               6759 non-null int64
leg_dist          6759 non-null float64
name              6759 non-null object
leg_nro           6759 non-null int64
orig_name         6759 non-null object
num_runs          6759 non-null float64
pred_log_mean     6725 non-null float64
pred_log_std      6725 non-null float64
dtypes: float64(4), int64(3), object(5)
memory usage: 633.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 4 columns):
first_name           943 non-null object
mean_pace_count      943 non-null int64
fn_pace_class        943 non-null int64
fn_pace_std_class    943 non-null int64
dtypes: int64(3), object(1)
memory usage: 29.6+ KB


2019-11-06 20:45:57,313 
None
2019-11-06 20:45:57,508     log_q_low  predicted  log_q_high  log_std
0       1.906      7.323       2.000    0.047
1       1.848      6.597       2.018    0.085
2       1.849      7.062       2.031    0.091
3       1.838      6.834       2.010    0.086
4       1.887      6.918       1.964    0.039
5       1.824      6.492       1.959    0.067
6       1.797      6.837       2.017    0.110
7       1.865      7.241       2.002    0.068
8       1.846      6.778       1.966    0.060
9       1.821      6.509       1.987    0.083
10      1.840      7.025       2.102    0.131
11      1.843      6.825       1.999    0.078
12      1.858      6.856       2.042    0.092
13      1.840      7.126       2.094    0.127
14      1.789      6.871       2.057    0.134
2019-11-06 20:45:57,511 0.17056615078015772


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6759 entries, 0 to 6758
Data columns (total 41 columns):
team_id                  -6759 non-null int64
team_id_log10            -6759 non-null float64
team_id_square           -6759 non-null int64
leg_1                    -6759 non-null Sparse[uint8, 0]
leg_2                    -6759 non-null Sparse[uint8, 0]
leg_3                    -6759 non-null Sparse[uint8, 0]
leg_4                    -6759 non-null Sparse[uint8, 0]
c_EST                    -6759 non-null Sparse[uint8, 0]
c_FIN                    -6759 non-null Sparse[uint8, 0]
c_GBR                    -6759 non-null Sparse[uint8, 0]
c_LAT                    -6759 non-null Sparse[uint8, 0]
c_NOR                    -6759 non-null Sparse[uint8, 0]
c_OTHER                  -6759 non-null Sparse[uint8, 0]
c_RUS                    -6759 non-null Sparse[uint8, 0]
c_SUI                    -6759 non-null Sparse[uint8, 0]
c_SWE                    -6759 non-null Sparse[uint8, 0]
runs_1.0    

2019-11-06 20:45:57,519 
count    6759.000000
mean        0.170566
std         0.044884
min        -0.104707
1%          0.037579
5%          0.083654
25%         0.148248
50%         0.175554
75%         0.201021
95%         0.232968
99%         0.254343
max         0.345540
Name: log_std, dtype: float64
2019-11-06 20:45:57,533 
   team_id                   team       team_base_name team_country  leg  \
0        1        Stora Tuna OK 1        Stora Tuna OK          SWE    1   
1        1        Stora Tuna OK 1        Stora Tuna OK          SWE    2   
2        1        Stora Tuna OK 1        Stora Tuna OK          SWE    3   
3        1        Stora Tuna OK 1        Stora Tuna OK          SWE    4   
4        2  Göteborg Majorna OK 1  Göteborg Majorna OK          SWE    1   

   leg_dist                name  leg_nro           orig_name  num_runs  \
0       6.0       tilda östberg        1       Tilda Östberg       1.0   
1       5.7    magdalena olsson        2    Magdalena Olsson   

CPU times: user 50.4 s, sys: 196 ms, total: 50.6 s
Wall time: 50.6 s


In [4]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)



(6759, 20)

In [5]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std", "num_runs"]]


In [6]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std,num_runs
0,1,Stora Tuna OK 1,1,6.0,Tilda Östberg,1.8857,0.067659,1.0
1,1,Stora Tuna OK 1,2,5.7,Magdalena Olsson,1.9587,0.082400,7.0
2,1,Stora Tuna OK 1,3,7.3,Julia Jakob,1.7824,0.090875,1.0
3,1,Stora Tuna OK 1,4,7.9,Tove Alexandersson,1.7963,0.067659,7.0
4,2,Göteborg Majorna OK 1,1,6.0,Amanda Hansson,2.0053,0.067659,2.0


In [7]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                                \
leg                          1                      2                      3   
team_id                                                                        
1              Stora Tuna OK 1        Stora Tuna OK 1        Stora Tuna OK 1   
2        Göteborg Majorna OK 1  Göteborg Majorna OK 1  Göteborg Majorna OK 1   
3                   MS Parma 1             MS Parma 1             MS Parma 1   
4          Järla Orientering 1    Järla Orientering 1    Järla Orientering 1   
5               Alfta ÖSA OK 1         Alfta ÖSA OK 1         Alfta ÖSA OK 1   

                               dist                               name  \
leg                          4    1    2    3    4                   1   
team_id                                                                  
1              Stora Tuna OK 1  6.0  5.7  7.3  7.9       Tilda Östberg   
2        Göteborg Majorna OK 1  6.0  5.7  7.3  7.9      Amanda Hansson   
3                   MS Parma 1  6.0  5.7  7.3  7.9   Anastasia Rudnaya   
4          Järla Orientering 1  6.0  5.7  7.3  7.9       Sara Sjökvist   
5               Alfta ÖSA OK 1  6.0  5.7  7.3  7.9  Galina Vinogradova   

                              ... log_mean           log_std            \
leg                        2  ...        3       4         1         2   
team_id                       ...                                        
1           Magdalena Olsson  ...   1.7824  1.7963  0.067659  0.082400   
2              Runa Fremstad  ...   1.9581  1.9854  0.067659  0.087900   
3             Anni Heikkonen  ...   1.8669  1.9113  0.067659  0.100600   
4        Anna Dahlgren-Rosén  ...   1.8086  1.9289  0.112800  0.182322   
5            Josefine Heikka  ...   1.9000  1.8720  0.102900  0.095440   

                            num_runs                 
leg             3         4        1    2    3    4  
team_id                                              
1        0.090875  0.067659      1.0  7.0  1.0  7.0  
2        0.067659  0.068116      2.0  5.0  5.0  3.0  
3        0.130894  0.067659      4.0  7.0  1.0  8.0  
4        0.133987  0.093145      4.0  0.0  1.0  2.0  
5        0.095300  0.112023      7.0  2.0  6.0  3.0  

[5 rows x 24 columns]

In [8]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1697, 24)

(1682, 24)

In [9]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [10]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [11]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,34.569260,71.913173,112.600482,158.703743,45.312026,88.912934,135.624634,185.071442,39.565228,80.054463,...,36.248523,41.578568,45.312026,47.437861,51.955521,54.362789,39.565228,40.325991,43.416831,47.654374
2,39.119883,78.253015,127.670548,183.198022,50.957859,97.070180,151.400325,211.203432,44.562518,87.134944,...,45.236309,50.301520,50.957859,50.403455,59.392919,65.767193,44.562518,42.447800,51.709364,57.604562
3,34.811820,70.940277,113.137726,164.775006,45.333162,89.416235,143.821490,198.621001,39.664004,79.514941,...,36.609760,46.677263,45.333162,48.357992,61.361936,60.976353,39.664004,39.749639,47.219513,53.395544
4,34.648182,68.773664,110.251934,162.522000,53.839090,104.352538,152.313206,208.800547,43.310191,84.293639,...,34.107657,45.198652,53.839090,58.339999,57.684205,65.302561,43.310191,40.577570,44.467241,54.375476
5,32.465902,67.469972,113.255956,161.281420,48.447414,89.115708,141.086208,196.904332,39.698567,77.540270,...,40.568928,40.923405,48.447414,45.461558,58.711153,63.849553,39.698567,37.653265,48.760407,51.342848


In [12]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4,num_runs_1,num_runs_2,num_runs_3,num_runs_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,1.78,1.80,0.07,0.08,0.09,0.07,1.0,7.0,1.0,7.0
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,1.96,1.99,0.07,0.09,0.07,0.07,2.0,5.0,5.0,3.0
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,1.87,1.91,0.07,0.10,0.13,0.07,4.0,7.0,1.0,8.0
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,1.81,1.93,0.11,0.18,0.13,0.09,4.0,0.0,1.0,2.0
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,1.90,1.87,0.10,0.10,0.10,0.11,7.0,2.0,6.0,3.0


In [13]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [14]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,36.248523,41.578568,45.312026,47.437861,51.955521,54.362789,39.565228,40.325991,43.416831,47.654374
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,45.236309,50.301520,50.957859,50.403455,59.392919,65.767193,44.562518,42.447800,51.709364,57.604562
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,36.609760,46.677263,45.333162,48.357992,61.361936,60.976353,39.664004,39.749639,47.219513,53.395544
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,34.107657,45.198652,53.839090,58.339999,57.684205,65.302561,43.310191,40.577570,44.467241,54.375476
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,40.568928,40.923405,48.447414,45.461558,58.711153,63.849553,39.698567,37.653265,48.760407,51.342848


In [15]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9587   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.7963   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   
        2    Göteborg Majorna OK 1   5.7       Runa Fremstad    2.0081   
        3    Göteborg Majorna OK 1   7.3    Kristine Berglia    1.9581   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.067659       1.0    34.569260   45.312026   39.565228   
        2    0.082400       7.0    71.913173   88.912934   80.054463   
        3    0.090875       1.0   112.600482  135.624634  123.524598   
        4    0.067659       7.0   158.703743  185.071442  171.365750   
2       1    0.067659       2.0    39.119883   50.957859   44.562518   
        2    0.087900       5.0    78.253015   97.070180   87.134944   
        3    0.067659       5.0   127.670548  151.400325  139.066372   

               fin_mean  fin_sum_log_mean  fin_sum_log_std  mass_start  \
team_id leg                                                              
1       1     39.659442          3.677973         0.068635         0.0   
        2     80.143840          4.382362         0.054058         0.0   
        3    123.743666          4.817095         0.047262         0.0   
        4    171.458161          5.143578         0.039032         0.0   
2       1     44.699516          3.797689         0.067413         0.0   
        2     87.273964          4.467521         0.055337         0.0   
        3    139.177594          4.934807         0.043441         0.0   

             dark_during_leg  personal_start95  personal_end95  \
team_id leg                                                      
1       1                0.0         34.569260       45.312026   
        2                0.0         34.236655       47.437861   
        3                0.0         36.248523       51.955521   
        4                0.0         41.578568       54.362789   
2       1                0.0         39.119883       50.957859   
        2                0.0         35.794020       50.403455   
        3                0.0         45.236309       59.392919   

             personal_median  
team_id leg                   
1       1          39.565228  
        2          40.325991  
        3          43.416831  
        4          47.654374  
2       1          44.562518  
        2          42.447800  
        3          51.709364

In [16]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [17]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]
local_start_ts = pd.Timestamp(start_timestamp).tz_localize(None)

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= local_start_ts)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9587   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.7963   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.067659       1.0    34.569260   45.312026   39.565228   
        2    0.082400       7.0    71.913173   88.912934   80.054463   
        3    0.090875       1.0   112.600482  135.624634  123.524598   
        4    0.067659       7.0   158.703743  185.071442  171.365750   
2       1    0.067659       2.0    39.119883   50.957859   44.562518   

               fin_mean  ...  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg              ...                                                 
1       1     39.659442  ...         0.068635         0.0              0.0   
        2     80.143840  ...         0.054058         0.0              0.0   
        3    123.743666  ...         0.047262         0.0              0.0   
        4    171.458161  ...         0.039032         0.0              0.0   
2       1     44.699516  ...         0.067413         0.0              0.0   

             personal_start95  personal_end95  personal_median  \
team_id leg                                                      
1       1           34.569260       45.312026        39.565228   
        2           34.236655       47.437861        40.325991   
        3           36.248523       51.955521        43.416831   
        4           41.578568       54.362789        47.654374   
2       1           39.119883       50.957859        44.562518   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
1       1   2019-06-15 14:39:33.913653374 2019-06-15 14:34:34.155590057   
        2   2019-06-15 15:20:03.267806530 2019-06-15 15:11:54.790393829   
        3   2019-06-15 16:03:31.475895882 2019-06-15 15:52:36.028941631   
        4   2019-06-15 16:51:21.944995165 2019-06-15 16:38:42.224599361   
2       1   2019-06-15 14:44:33.751105309 2019-06-15 14:39:07.192989349   

                           fin_time_end95 personal_estimate_interval  
team_id leg                                                           
1       1   2019-06-15 14:45:18.721583366                  10.742767  
        2   2019-06-15 15:28:54.776010513                  13.201206  
        3   2019-06-15 16:15:37.478013277                  15.706998  
        4   2019-06-15 17:05:04.286492586                  12.784221  
2       1   2019-06-15 14:50:57.471560240                  11.837976  

[5 rows x 21 columns]

In [18]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

for_print[for_print['team'].str.contains("Stadin")]
#for_print = for_print[for_print['team'].str.contains("Puskasilimät")]

for_print = for_print[[
 'leg',
 'name', 
 'num_runs', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,num_runs,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,,
1,1,Tilda Östberg,1.0,34.57,39.57,45.31,10.74
1,2,Magdalena Olsson,7.0,34.24,40.33,47.44,13.20
1,3,Julia Jakob,1.0,36.25,43.42,51.96,15.71
1,4,Tove Alexandersson,7.0,41.58,47.65,54.36,12.78
2,1,Amanda Hansson,2.0,39.12,44.56,50.96,11.84
...,...,...,...,...,...,...,...
1722,4,Johanna Liinamaa,3.0,78.39,89.43,101.78,23.39
1723,1,Elina Holstikko,0.0,62.85,89.37,129.35,66.50
1723,2,Mirka Leinonen,0.0,59.82,85.51,122.89,63.06


In [19]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'num_runs',     
 'ind_log_mean', 
 'ind_log_std',
 'personal_start95', 
 'personal_median',
 'personal_end95',    
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Downt")]

,team_id,leg,team,name,num_runs,ind_log_mean,ind_log_std,personal_start95,personal_median,personal_end95,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
1332,341,1,Rasti-Jyry 1,Jola Maisala,8.0,2.188600,0.126500,41.752580,53.436046,68.596148,3.978328,0.126046,2019-06-15 11:41:45.154812098+00:00,2019-06-15 12:08:35.768897057+00:00,2019-06-15 11:53:26.162764072+00:00,0.0000,0.0
1333,341,2,Rasti-Jyry 1,Kaija Rantoja,7.0,2.296600,0.144100,42.902361,56.649653,75.411115,4.706293,0.097334,2019-06-15 12:31:42.663934946+00:00,2019-06-15 13:14:07.830876827+00:00,2019-06-15 12:50:31.483548403+00:00,0.0000,0.0
1334,341,3,Rasti-Jyry 1,Jenni Castren,5.0,2.334200,0.141700,56.690995,75.637749,99.343734,5.229579,0.082233,2019-06-15 13:38:44.556781292+00:00,2019-06-15 14:38:49.090376377+00:00,2019-06-15 14:06:40.775005102+00:00,0.0000,0.0
1335,341,4,Rasti-Jyry 1,Inka Lemmetyinen,1.0,2.293200,0.184172,54.907744,78.209625,111.500555,5.583882,0.079318,2019-06-15 14:48:23.837378740+00:00,2019-06-15 16:11:28.261754036+00:00,2019-06-15 15:25:39.413773060+00:00,0.0000,0.0
2076,528,1,Rasti-Jyry 2,Päivi Taponen,3.0,2.361600,0.166955,46.353963,63.679319,88.051172,4.155268,0.165947,2019-06-15 11:46:21.237798929+00:00,2019-06-15 12:28:03.070313692+00:00,2019-06-15 12:03:40.759133577+00:00,0.0000,0.0
2077,528,2,Rasti-Jyry 2,Mervi Luurila,2.0,2.473200,0.190174,46.105526,67.604129,98.934635,4.886201,0.127916,2019-06-15 12:43:22.242751122+00:00,2019-06-15 13:49:57.940974712+00:00,2019-06-15 13:12:27.141387701+00:00,0.0000,0.0
2078,528,3,Rasti-Jyry 2,Suvi Tiainen,2.0,2.646600,0.193850,70.251659,102.957034,149.486477,5.467129,0.109976,2019-06-15 14:11:49.468293667+00:00,2019-06-15 15:53:50.284385681+00:00,2019-06-15 14:56:30.984000206+00:00,0.0000,0.0
2079,528,4,Rasti-Jyry 2,Tiina Isoniemi,6.0,2.153800,0.067659,59.643380,67.981437,77.645074,5.721134,0.086643,2019-06-15 15:18:05.484685898+00:00,2019-06-15 17:02:03.322562933+00:00,2019-06-15 16:04:53.602000475+00:00,0.1163,0.0
2916,740,1,Rasti-Jyry 4,Mila Viksilä,0.0,2.342982,0.182322,43.726321,62.325855,89.438815,4.135063,0.182942,2019-06-15 11:43:43.579284191+00:00,2019-06-15 12:29:26.328924179+00:00,2019-06-15 12:02:19.551310539+00:00,0.0000,0.0
2917,740,2,Rasti-Jyry 4,Tiina Sidensnöre,7.0,2.262600,0.146600,41.258084,54.733049,73.405667,4.771309,0.120682,2019-06-15 12:33:44.890198708+00:00,2019-06-15 13:30:27.279522896+00:00,2019-06-15 12:58:11.280007362+00:00,0.0000,0.0


In [20]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [21]:
for_web.shape

(6728, 17)

In [22]:
for_print[['num_runs', 'personal_median', "personal_estimate_interval"]].groupby('num_runs').agg(["mean", "count"]).round(2)


personal_median       personal_estimate_interval      
                    mean count                       mean count
num_runs                                                       
0.0                82.81  1647                      66.53  1647
1.0                80.71  1008                      61.65  1008
2.0                81.55   725                      57.62   725
3.0                78.13   694                      54.49   694
4.0                76.99   563                      32.77   563
5.0                75.32   550                      34.88   550
6.0                73.29   554                      33.80   554
7.0                71.36   575                      32.86   575
8.0                71.38   409                      32.37   409
9.0                83.95     2                      36.03     2
10.0               62.39     1                      16.80     1

In [23]:
for_web["interval_length"] = for_web.fin_time_end95 - for_web.fin_time_start95
for_web["interval_length"].median()
#for_web[['leg', 'interval_length', "num_runs"]].info()

Timedelta('0 days 01:09:24.787935')

In [24]:
for_web[['leg', 'interval_length', "num_runs"]].groupby('leg').agg(["mean", "count"]).round(2)

num_runs      
        mean count
leg               
1       2.99  1682
2       2.76  1682
3       3.00  1682
4       3.25  1682

In [25]:
shared.log_df(f"{ve_or_ju} DONE")

2019-11-06 20:47:17,423 ve DONE
